# <img src ='https://airsblobstorage.blob.core.windows.net/airstream/Asset 275.png' width="50px"> Access Azure Blob Storage from Azure Databricks using Azure Key Vault 

[link to tutorial](https://docs.microsoft.com/en-us/azure/databricks/scenarios/store-secrets-azure-key-vault)

Run the following command to mount your blob storage container. Remember to change the values for the following properties:

* your-container-name
* your-storage-account-name
* mount-name
* config-key
* scope-name
* key-name

In [0]:
#list all secret scopes 
dbutils.secrets.listScopes()

In [0]:
dbutils.secrets.list("akv-bck-scope")

In [0]:
dbutils.secrets.get(scope="akv-bck-scope", key="ApplicationInsights--ConnectionString")

In [0]:
dbutils.secrets.list("databricks-bck-scope")

In [0]:
%scala

val driverClass = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
val connectionProperties = new java.util.Properties()
connectionProperties.setProperty("Driver", driverClass)

val jdbcUsername = dbutils.secrets.get(scope = "databricks-bck-scope", key = "username")
val jdbcPassword = dbutils.secrets.get(scope = "databricks-bck-scope", key = "password")
val ServicePrincipal = dbutils.secrets.get(scope = "databricks-bck-scope", key = "ServicePrincipal")
val ServicePrincipalPassword = dbutils.secrets.get(scope = "databricks-bck-scope", key = "ServicePrincipalPassword")

In [0]:
value = dbutils.secrets.get(scope="databricks-bck-scope", key="ServicePrincipalPassword")

for char in value:
    print(char, end=" ")

In [0]:
dbutils.fs.mount(
source = "wasbs://apollo@airsblobstorage.blob.core.windows.net",
mount_point = "/mnt/apollo",
extra_configs = {"fs.azure.account.key.airsblobstorage.blob.core.windows.net":dbutils.secrets.get(scope = "akv-bck-scope", key = "CloudStorage--AccountKey")})

Run the following command to read a file in your blob storage container to a dataframe. Change the values in the command to match your mount name and file name.

In [0]:
df = spark.read.text("/mnt/apollo/apollo11.json")

Use the following command to display the contents of your file.

In [0]:
df.show()

To unmount your blob storage, run the following command

In [0]:
dbutils.fs.unmount("/mnt/apollo")

Notice that once the mount has been unmounted, you can no longer read from your blob storage account